In [5]:
#Importing
from datasets import load_dataset
from transformers import GPT2Tokenizer

#Creating a n object for  GPT2Tokenizer. 
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

#Loading the textfile as datset
dataset = load_dataset("text", data_files={"train": "my_dataset.txt"})

#Assigning the Token ids to all the Tokens.
def tokenize(batch):
    tokens = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

#Removing the text columns
tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])
print(tokenized_dataset)

Map:   0%|          | 0/8 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8
    })
})


In [6]:
#Importing the actual model
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

#Model is created
model = GPT2LMHeadModel.from_pretrained("gpt2")

#Giving the training Arguments.
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    eval_strategy="no",
)

#Training the Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss


TrainOutput(global_step=6, training_loss=3.726496696472168, metrics={'train_runtime': 32.6759, 'train_samples_per_second': 0.734, 'train_steps_per_second': 0.184, 'total_flos': 1567752192000.0, 'train_loss': 3.726496696472168, 'epoch': 3.0})

In [17]:
from transformers import pipeline

#Generating the teaxt
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
output = generator("The stars began to", max_new_tokens=50)
print(output[0]['generated_text'])


Device set to use cpu


The stars began to be more and the stars in the room, the stars stars began to form.


The star was very very thin and it was very very very thin.


The stars were looking in the middle of the stars.


The stars from
